Below, we create some useful functions and code-bits.

In [ ]:

#-----Import packages-----#
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft, fftfreq #Documentation: https://docs.scipy.org/doc/scipy/reference/fft.html#module-scipy.fft
from scipy.signal import find_peaks
import numpy as np
from numpy import real, pi


#-----Define functions-----#

def print_vars(**kwargs):
    output = "; ".join([f"{name} = {value}" for name, value in kwargs.items()])
    print(output)

def myfft(signal, timeaxis, fs):
    #function calculates fft and ifft given signal
  signal_fft = (fft(signal, norm="forward")) #calculate forward (normalized) fft. in general, will be complex
  signal_ifft = real(ifft(signal_fft, norm="forward")) #invert the fft to hopefully recreate original signal. note that you cannot ignore phase !
  freqaxis = fftfreq(len(signal),d=1/fs) #frequency x-axis for fft
  return signal_fft, signal_ifft, freqaxis

def plotmyfft(signal, timeaxis, signal_fft, signal_ifft, freqaxis, fs): 
    #function plots signal, fft and ifft
  fig,axs = plt.subplots(nrows = 3,
                         ncols = 1,
                         constrained_layout="true")
  
  ax=axs[0]
  ax.plot(timeaxis,signal) #plot signal
  ax.set_xlabel("Time (s)")
  ax.set_ylabel("Signal")
  ax.set_title("Signal")

  ax=axs[1]
  fft_abs=abs(signal_fft) #take the absolute value (aka magnitude) of the fft
  ax.plot(freqaxis,fft_abs) #plot fft magnitude. ignoring phase here
  peaks, _ = find_peaks(fft_abs, height=0.4) # You might need to adjust the height parameter based on your data
  ax.scatter(freqaxis[peaks], fft_abs[peaks], color='red') # mark peak points
  # Print peaks
  for peak in peaks:
    print_vars(peak=freqaxis[peak],magnitude=fft_abs[peak])
  ax.set_xlabel("Frequency (Hz)")
  #ax.set_xlim([-100, 100])
  ax.set_ylabel("Magnitude")
  ax.set_title("FFT of Signal")

  ax=axs[2]
  ax.plot(timeaxis,signal_ifft) #plot recovered time signal
  ax.set_xlabel("Time (s)")
  ax.set_ylabel("Recovered Signal")
  ax.set_title("Inverse FFT of Signal (Recovered signal)")

  return

def plotmyfft2(signal, timeaxis, signal_fft, freqaxis, fs):
    #function plots signal and fft. 
  fig,axs = plt.subplots(nrows = 2,
                         ncols = 1,
                         constrained_layout="true")
  
  ax=axs[0]
  ax.plot(timeaxis,signal) #make plots
  ax.set_xlabel("Time (s)")
  ax.set_ylabel("Signal")
  ax.set_title("Signal")

  ax=axs[1]
  fft_abs=abs(signal_fft) #take the absolute value (aka magnitude) of the fft
  ax.plot(freqaxis,fft_abs) #plot fft magnitude. ignoring phase here
  peaks, _ = find_peaks(fft_abs, height=0.05) # You might need to adjust the height parameter based on your data
  ax.scatter(freqaxis[peaks], fft_abs[peaks], color='red') # mark peak points

  # Print peaks
  for peak in peaks:
    print_vars(peak=freqaxis[peak],magnitude=fft_abs[peak])

  ax.set_xlabel("Frequency (Hz)")
  # ax.set_xlim([-100, 100])
  ax.set_ylabel("Magnitude")
  ax.set_title("FFT of Signal")

Then we generate two sine-waves and look at their magnitude spectra.

In [ ]:
plt.rcParams['figure.figsize'] = [6, 6] #set figure dimensions

fs = 1000.0 #sampling frequency
timeaxis = np.arange(0,2.0,step=1/fs)

freq = 30 #sine-wave frequency
amp = 1 #amplitude
signal1 = amp * np.sin(2.0*pi*freq*timeaxis) #create sine wave

signal1_fft, signal1_ifft, freqaxis =  myfft(signal1, timeaxis, fs)
plotmyfft(signal1, timeaxis, signal1_fft, signal1_ifft, freqaxis, fs)
vals1 = find_peaks(abs(signal1_fft),0.04)
print('--------------------------------------------------')

In [ ]:
#change sine-wave frequency and amplitude. identify and interpret effects on plot


freq=10 #sine-wave frequency
amp=10 #amplitude
signal2 = amp * np.sin(2.0*pi*freq*timeaxis) #create sine wave

signal2_fft, signal2_ifft, freqaxis =  myfft(signal2, timeaxis, fs)
plotmyfft(signal2, timeaxis, signal2_fft, signal2_ifft, freqaxis, fs)
vals2 = find_peaks(abs(signal2_fft),0.04)
print('--------------------------------------------------')


Now using the code above as a template - experiment with the frequency content of signals. For example:

1. What happened to the Fourier transform (FT) when the amplitude and frequency of the sine-wave were changed ? Why ?
2. Try other values for frequency and amplitude and confirm.
3. Try adding a phase to the sine-wave. What happens to the plotted FT ?
4. Add two sine-waves - what happens to the frequency spectrum ? Change the amplitude of each sine-wave and comment on what happens
5. Go back to a single sine-wave. Rectify it - this is a non-linear operation (why ?). What happens to the frequency spectrum ?

Now take a look at a couple of new signals and their Fourier transforms.

Signal 1 - acts as a low-pass filter. What does this mean ? And why ? Explain in the time and frequency domains.

In [ ]:
timeaxis2=np.arange(0,0.15,step=1/fs)# Time values from 0 to 500 milliseconds
alpha = 5
beta = 250
y = (1 - np.exp(-alpha * timeaxis2)) * np.exp(-beta * (timeaxis2 - 0.1))
y=y/max(y)

y_fft, y_ifft, y_freqaxis =  myfft(y, timeaxis2, fs)
plotmyfft2(y, timeaxis2, y_fft, y_freqaxis, fs)


Let us generate a sine-wave. Filter it by y, either by convolution in the time-domain or multiplication in the frequency domain (followed by an ifft). Do they give the same response ? Look carefully at the end of the output.. why does the output extend beyond 2 s ?

In [ ]:

freq1=30 #sine-wave frequency
amp1=1 #amplitude
signal1=amp1*np.sin(2.0*pi*freq1*timeaxis) #create sine wave
signal1_fft, signal1_ifft, freqaxis =  myfft(signal1, timeaxis, fs)

# Time-domain convolution
result_time_domain = np.convolve(signal1, y, mode='full')

# Frequency-domain multiplication
N = len(signal1) + len(y) - 1  # this is the size after convolution
X = fft(signal1, n=N)
Y = fft(y, n=N)
result_frequency_domain = real(ifft(X * Y)) #you could check that the imaginary parts are just very small numerical errors

# Plotting the results
plt.figure(figsize=(10, 4))
plt.plot((1/fs)*np.arange(0,len(result_time_domain)),
         result_time_domain, 
         label='Time-domain convolution', 
         color='blue')
plt.plot((1/fs)*np.arange(0,len(result_frequency_domain)),
         result_frequency_domain, 
         label='Frequency-domain multiplication', 
         color='red', 
         linestyle='--')
plt.title("Comparison of Convolution Methods")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# To ensure that both methods produce nearly the same result
# (There can be very tiny differences due to numerical precision)
assert np.allclose(result_time_domain, result_frequency_domain, atol=1e-5), "The results are not the same!"


Note that this similarity may be artificial, since np.conv may be using fft under the hood - I am not sure. For longer signals, and with naive implementations, a method based on fft will be much faster.

Now let us test that low-pass filter claim a bit. First generate a sine-wave with different frequencies: 30 Hz and 200 Hz. Filter it by y. Before running the code, what do you expect ? Is the prediction easier from the time-domain or from the frequency domain ?

In [ ]:
freq1=30 #sine-wave frequency
amp1=1 #amplitude
signal1=amp1*np.sin(2.0*pi*freq1*timeaxis) #create sine wave
output1 = np.convolve(signal1, y, mode='full')

freq2=200#sine-wave frequency
amp2=1 #amplitude
signal2=amp2*np.sin(2.0*pi*freq2*timeaxis)  #create sine wave
output2 = np.convolve(signal2, y, mode='full')

plt.figure(figsize=(10, 4))
plt.plot((1/fs)*np.arange(0,len(output1)),
         output1, 
         label=str(freq1)+" Hz", 
         color='blue')
plt.plot((1/fs)*np.arange(0,len(output2)),
         output2, 
         label=str(freq2) + "Hz", 
         color='red', 
         linestyle='--')
plt.title("Comparison of filter outputs")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.legend()
plt.grid(True)
plt.xlim([0,0.2])
plt.tight_layout()
plt.show()

Now do the same thing as above, but with some other sine-wave frequencies. Stay well below 500 Hz, to avoid aliasing. If you want to use higher frequencies, what needs to change ?

Then try a sum of two sinusoids, like 30 Hz and 200 Hz. What does the output look like ? Why ? Plot both the output signal and its Fourier transform (using the code towards the top of this notebook). Interpret it.

In [ ]:
#insert code here to test other frequencies and the sum of two sinusoids.

Finally, let us try a different filter. Try to predict its effect on the same sinusoids at 30 Hz and 200 Hz, and the sum of the two.

More challenging: why does this filter, from its time-domain properties (and definition in code) have a spectrum that looks as it does ? Can you intuitively understand this ?

In [ ]:
plt.rcParams['figure.figsize'] = [6, 4]

center_freq=200
y2=y*np.sin(2.0*pi*center_freq*timeaxis2) #create sine wave, multiplied by the previous filter y

y2_fft, y2_ifft, y2_freqaxis =  myfft(y2, timeaxis2, fs)
plotmyfft2(y2, timeaxis2, y2_fft, y2_freqaxis, fs)

Now let us check the prediction for the 30 Hz and 200 Hz sinusoids and their sum. Write your code, based on the template above for the low-pass filter. Interpret the results.

In [ ]:
#Insert code here to do the above